In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

hp = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

hp.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [5]:
import scipy.stats as stats
from scipy.stats.mstats import winsorize
#add winsorized variables to original dataframe
hp['wins_grlivarea'] = winsorize(hp['grlivarea'], (0, 0.05))
hp['wins_garagearea'] = winsorize(hp['garagearea'], (0, 0.05))
hp['wins_totalbsmtsf'] = winsorize(hp['totalbsmtsf'], (0, 0.05))
hp['wins_firstflrsf'] = winsorize(hp['firstflrsf'], (0, 0.05))

In [8]:
# Y is the target variable
Y = hp['saleprice']

# X is the feature set 
X = hp[['overallqual', 'fullbath', 'garagecars', 'wins_totalbsmtsf', 'wins_firstflrsf', 'wins_grlivarea','wins_garagearea']]

lrm = linear_model.LinearRegression()

lrm.fit(X, Y)

print('\nCoefficients: \n', lrm.coef_)
print('\nIntercept: \n', lrm.intercept_)


Coefficients: 
 [2.38524512e+04 1.61369838e+03 8.20389341e+03 3.23355847e+01
 1.60425862e+01 4.67026909e+01 3.30621135e+01]

Intercept: 
 -118869.09625681039


In [7]:
import statsmodels.api as sm

# We need to manually add a constant
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     662.0
Date:                Thu, 17 Oct 2019   Prob (F-statistic):               0.00
Time:                        10:11:26   Log-Likelihood:                -17498.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1452   BIC:                         3.505e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.189e+05   4998.934    -23.779      0.000   -1.29e+05   -1.09e+05
overallqual       2.385e+04   1108.134     21.525      0.000    2.17e+04     2.6e+04
fullbath          1613.6984   2535.776      0.636      0.525   -3360.478    6587.875
garagecars        8203.8934   3209.332      2.556      0.011    1908.470    1.45e+04
wins_totalbsmtsf    32.3356      4.687      6.899      0.000      23.141      41.530
wins_firstflrsf     16.0426      5.319      3.016      0.003       5.609      26.476
wins_grlivarea      46.7027      3.339     13.987      0.000      40.153      53.253
wins_garagearea     33.0621     11.666      2.834      0.005      10.179      55.945
==============================================================================
Omnibus:                      786.379   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19210.087
Skew:                           1.995   Prob(JB):                         0.00
Kurtosis:                      20.316   Cond. No.                     1.13e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Looking at the model, all of the variables are statistically significant except for fullbath so we will remove that one. Because we found earlier that garagecars and wins_garagearea are correlated, we will also remove garagecars from the model. 

In [9]:
# Y is the target variable
Y = hp['saleprice']

# X is the feature set 
X = hp[['overallqual', 'wins_totalbsmtsf', 'wins_firstflrsf', 'wins_grlivarea','wins_garagearea']]

X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     921.9
Date:                Thu, 17 Oct 2019   Prob (F-statistic):               0.00
Time:                        10:15:32   Log-Likelihood:                -17502.
No. Observations:                1460   AIC:                         3.502e+04
Df Residuals:                    1454   BIC:                         3.505e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.195e+05   4994.986    -23.932      0.000   -1.29e+05    -1.1e+05
overallqual       2.453e+04   1069.656     22.937      0.000    2.24e+04    2.66e+04
wins_totalbsmtsf    32.0277      4.693      6.824      0.000      22.821      41.234
wins_firstflrsf     15.9621      5.328      2.996      0.003       5.511      26.413
wins_grlivarea      48.1500      3.029     15.896      0.000      42.208      54.092
wins_garagearea     58.2348      6.518      8.935      0.000      45.449      71.020
==============================================================================
Omnibus:                      778.135   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19031.741
Skew:                           1.965   Prob(JB):                         0.00
Kurtosis:                      20.246   Cond. No.                     1.12e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

After removing the two features, the rest of the the features maintained similar coefficients as before except for the wins_garagearea which increased from around 33 to 58. They also all remained statistically significant. The most prominent feature is overallqual which is probably to be expected. For every 1 point change in this variable, there is a 24,530 increase in saleprice. The other variables have a range between 15.96 to 58.23 increase in saleprice for every additional point gain. Overall, I think these results seem reasonable as we would expect that the actual quality of the house would be the biggest predictor as to the saleprice.  